In [1]:
import numpy as np;
import pandas as pd;

## DQL

In [2]:
cities = {
    'population':[1.9,0.31,1.84],
    'area':[1484,331,603],
    'state':['Delhi','Maharashtra','Maharashtra']
}

cities_frame = pd.DataFrame(cities,index=['Delhi','Pune','Mumbai'])



In [3]:
# Each column is a series

print(type(cities_frame.area))
cities_frame.area


<class 'pandas.core.series.Series'>


Delhi     1484
Pune       331
Mumbai     603
Name: area, dtype: int64

In [4]:
# Each index is also a series (with columns as index)

print(type(cities_frame.loc['Delhi']))
print(cities_frame.loc['Delhi'].index)
cities_frame.loc['Delhi']

<class 'pandas.core.series.Series'>
Index(['area', 'population', 'state'], dtype='object')


area           1484
population      1.9
state         Delhi
Name: Delhi, dtype: object

### Note :  
Important to understand that the operations on dataFrames give back another dataFrames/Series. And further operations can be applied on the results (chaining)

#### Projection

In [5]:
cities_frame['area']

Delhi     1484
Pune       331
Mumbai     603
Name: area, dtype: int64

In [6]:
cities_frame.area # same as above

Delhi     1484
Pune       331
Mumbai     603
Name: area, dtype: int64

In [7]:
cities_frame[['area','population']] # select area,population from cities

,area,population
Delhi,1484,1.90
Pune,331,0.31
Mumbai,603,1.84


In [8]:
# Apply function on each row : apply is done on the series
cities_frame.population.apply(np.log10) # select log10(population) from cities

Delhi     0.278754
Pune     -0.508638
Mumbai    0.264818
Name: population, dtype: float64

In [9]:
# population density
# select population*100000/area from cities where index in ('Delhi','Pune')
(cities_frame.population*100000/cities_frame.area)[['Delhi','Pune']]

Delhi    128.032345
Pune      93.655589
dtype: float64

### Note : 
<ul>apply : the provided function works on each index(row) of the dataFrame
<li>if the result of the operation on each index is a scalar, 'apply' outputs a series
<li>if the result of the operation on each index is a series, 'apply' outputs a dataFrame with each vector result represented as series as index.
<li>if the result of the operation on each index is a list (of same cardinality as each index in the dataFrame), 'apply' outputs a dataFrame with each vector result represented as series as index.
</ul>

In [10]:
# The output of lambda is the modified series, so final output of apply is a dataFrame
cities_frame.apply(lambda series : series.apply(lambda value : value.upper() if type(value)==str else value),axis=1)

,area,population,state
Delhi,1484,1.90,DELHI
Pune,331,0.31,MAHARASHTRA
Mumbai,603,1.84,MAHARASHTRA


In [11]:
# [0,0,None] has same cardinality(3) as Delhi,Pune and Mumbai
cities_frame.apply(lambda series : [0,0,None],axis=1)

,area,population,state
Delhi,0.0,0.0,NaN
Pune,0.0,0.0,NaN
Mumbai,0.0,0.0,NaN


In [12]:
# [0,0,0,0] has different cardinality(4) than Delhi,Pune and Mumbai
cities_frame.apply(lambda series : [0,0,0,0],axis=1)

Delhi     [0, 0, 0, 0]
Pune      [0, 0, 0, 0]
Mumbai    [0, 0, 0, 0]
dtype: object

In [13]:
# This is same as above example of population density
# select population*100000/area from cities where index in ('Delhi','Pune')
cities_frame.apply(lambda series : series.population*100000/series.area,axis=1)[['Delhi','Pune']]

Delhi    128.032345
Pune      93.655589
dtype: float64

In [14]:
# select area||' sq.kms',population||' lakhs' from cities

cities_frame[['area','population']] \
.apply(lambda series : [str(series.area)+' sq.kms',str(series.population)+' lakhs'],axis=1)

,area,population
Delhi,1484.0 sq.kms,1.9 lakhs
Pune,331.0 sq.kms,0.31 lakhs
Mumbai,603.0 sq.kms,1.84 lakhs


In [15]:
cities_frame[['area','population']]  \
.apply(lambda series : series.area,axis=1)

Delhi     1484.0
Pune       331.0
Mumbai     603.0
dtype: float64

#### Selection (where clause) 

In [33]:
cities_frame.loc['Delhi']  # Where index='Delhi'

area           1484
population      1.9
state         Delhi
Name: Delhi, dtype: object

In [34]:
cities_frame.loc['Delhi':'Pune']  # index = 'Delhi' to 'Pune'

,area,population,state
Delhi,1484,1.90,Delhi
Pune,331,0.31,Maharashtra


In [17]:
cities_frame.loc[['Delhi','Pune']] # Where index in ('Delhi','Pune')

,area,population,state
Delhi,1484,1.90,Delhi
Pune,331,0.31,Maharashtra


In [18]:
cities_frame.loc[cities_frame.area>600] # Note : cities_frame.area>600 is a bool series

,area,population,state
Delhi,1484,1.90,Delhi
Mumbai,603,1.84,Maharashtra


#### Projection + Selection

In [19]:
cities_frame[['state','population']].loc[cities_frame.area>600] # select state,population from cities where area>600

,state,population
Delhi,Delhi,1.90
Mumbai,Maharashtra,1.84


In [20]:
cities_frame.loc[cities_frame.area>600][['state','population']] # select state,population from cities where area>600

,state,population
Delhi,Delhi,1.90
Mumbai,Maharashtra,1.84


In [21]:
# select area from cities where name = 'Delhi'
cities_frame.loc['Delhi','area'] # Note first param is index and second param is column

1484

In [22]:
# select area from cities where name = 'Delhi'
cities_frame.at['Delhi','area']

1484

#### Note : Use 'at' if you only need to get or set a single value in a DataFrame or Series." 
#### loc on the other hand can be used to access a single value but also to access a group 
#### of rows and columns by a label or labels.
#### When it comes to speed the answer is clear: we should definitely use at.

In [23]:
# select area where name in ('Delhi','Mumbai')
cities_frame.loc[['Delhi','Mumbai'],'area']

Delhi     1484
Mumbai     603
Name: area, dtype: int64

In [24]:
# select state, population from cities where area > 600
cities_frame.loc[cities_frame.area>600,['state','population']]

,state,population
Delhi,Delhi,1.90
Mumbai,Maharashtra,1.84


In [25]:
# Nested query
# select * from cities where area>600 (select * from cities where state='Maharashtra')
cities_frame.loc[cities_frame.area>600].loc[cities_frame.state=='Maharashtra'] 

,area,population,state
Mumbai,603,1.84,Maharashtra


#### Order by

In [26]:
# select state,population from cities order by state desc, population asc

cities_frame.sort_values(by=['state','population'],ascending=[False,True])[['state','population']]

,state,population
Pune,Maharashtra,0.31
Mumbai,Maharashtra,1.84
Delhi,Delhi,1.90


#### Aggregation

In [35]:
cities_frame.area.sum() # select sum(area) from cities

2418

In [36]:
cities_frame['area'].aggregate(sum) # same as above

2418

In [28]:
np.mean(cities_frame.population) # select avg(area) from cities

1.3499999999999999

In [29]:
cities_frame.area.max(),cities_frame.population.min() # select max(area),min(population) from cities

(1484, 0.31)